In [ ]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras import Sequential

In [3]:
train      = pd.read_csv("/content/drive/MyDrive/github_commit/sooktat_DL_project/dataset/train_data.csv")
test       = pd.read_csv("/content/drive/MyDrive/github_commit/sooktat_DL_project/dataset/test_data.csv")
submission = pd.read_csv("/content/drive/MyDrive/github_commit/sooktat_DL_project/dataset/sample_submission.csv")
topic_dict = pd.read_csv("/content/drive/MyDrive/github_commit/sooktat_DL_project/dataset/topic_dict.csv")

# BASELINE code 에서의 전처리 코드
## 1) Simple_Dense_Layer_Model

In [ ]:
# ------ 코드분석 (1) -------
def clean_text(sent):
  sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", " ", sent)
  return sent_clean

# ------ 코드분석 (2) -------
train["cleaned_title"] = train["title"].apply(lambda x : clean_text(x))
test["cleaned_title"]  = test["title"].apply(lambda x : clean_text(x))

train_text = train["cleaned_title"].tolist() # .tolist() : series 형식으로 된 데이터를 list로
test_text = test["cleaned_title"].tolist()
train_label = np.asarray(train.topic_idx) # np.asarray() : series 형식으로 된 데이터를 array로

# ------ 코드분석 (3) -------
tfidf = TfidfVectorizer(analyzer='word', sublinear_tf=True, ngram_range=(1, 2), max_features=150000, binary=False)

tfidf.fit(train_text)

train_tf_text = tfidf.transform(train_text).astype('float32')
test_tf_text  = tfidf.transform(test_text).astype('float32')

### 코드분석 (1)


```
re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", " ", sent)
```
<br/>

1.   정규표현식


|제목|내용|
|:---:|---|
| [^xy] |not 을 표현하며  x 및 y 를 제외한 문자를 의미한다.|
|[x-z]|	range를 표현하며 x ~ z 사이의 문자를 의미한다. |
|\\s|space 를 표현하며 공백 문자를 의미한다. |

<br/>

[^가-힣ㄱ-ㅎㅏ-ㅣ\\s] 의미   
: [ '가'부터 '힣'까지 (모음자음병합어) + 'ㄱ'부터 'ㅎ'까지 (자음) + 'ㅏ'부터 'ㅣ'까지 (모음) + 공백(blank) ] 에 해당되는 문자로 시작하는 문자열을 제외한 모든 문자열

<br/>

2. re.sub()

re.sub（정규 표현식, 치환 문자, 대상 문자열）  
: 정규 표현식에 해당되는 문자들을 지정한 문자로 치환

ex.

```
import re  

text = "I like apble And abple" 
text_mod = re.sub('apble|abple',"apple",text) 

print (text_mod)
```

<br/>

즉, 상기 코드의 의미  
: sent라는 문자 데이터에서 / '가'부터 '힣'까지 (모음자음병합어) + 'ㄱ'부터 'ㅎ'까지 (자음) + 'ㅏ'부터 'ㅣ'까지 (모음) + 공백(blank) 에 해당되는 모든 문자로 시작되는 문자열들을 제외한 모든 문자열을 / 공백(blank)로 치환한다.

### 코드분석 (2)



```
train["cleaned_title"] = train["title"].apply(lambda x : clean_text(x))
test["cleaned_title"]  = test["title"].apply(lambda x : clean_text(x))

train_text = train["cleaned_title"].tolist() # .tolist() : series 형식으로 된 데이터를 list로
test_text = test["cleaned_title"].tolist()
train_label = np.asarray(train.topic_idx) # np.asarray() : series 형식으로 된 데이터를 array로
```



In [ ]:
train['title']

0                  인천→핀란드 항공기 결항…휴가철 여행객 분통
1            실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화
2            이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것
3          NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합
4                 시진핑 트럼프에 중미 무역협상 조속 타결 희망
                        ...                
45649          KB금융 미국 IB 스티펠과 제휴…선진국 시장 공략
45650       1보 서울시교육청 신종코로나 확산에 개학 연기·휴업 검토
45651           게시판 키움증권 2020 키움 영웅전 실전투자대회
45652                     답변하는 배기동 국립중앙박물관장
45653    2020 한국인터넷기자상 시상식 내달 1일 개최…특별상 김성후
Name: title, Length: 45654, dtype: object

In [ ]:
train['cleaned_title']
# 첫번째 줄의 '->' '...' 가 공백으로
# 네번째 줄의 'NYT', '韓', '...' 가 공백으로
# 문자 전처리가 잘 되었음.

0                  인천 핀란드 항공기 결항 휴가철 여행객 분통
1            실리콘밸리 넘어서겠다 구글   조원 들여  전역 거점화
2            이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것
3              클린턴 측근 기업 특수관계 조명 공과 사 맞물려종합
4                 시진핑 트럼프에 중미 무역협상 조속 타결 희망
                        ...                
45649            금융 미국    스티펠과 제휴 선진국 시장 공략
45650        보 서울시교육청 신종코로나 확산에 개학 연기 휴업 검토
45651           게시판 키움증권      키움 영웅전 실전투자대회
45652                     답변하는 배기동 국립중앙박물관장
45653         한국인터넷기자상 시상식 내달  일 개최 특별상 김성후
Name: cleaned_title, Length: 45654, dtype: object

In [ ]:
train_text # list로 저장됨

In [ ]:
train.topic_idx # array로 저장됨

0        4
1        4
2        4
3        4
4        4
        ..
45649    1
45650    2
45651    1
45652    2
45653    2
Name: topic_idx, Length: 45654, dtype: int64

In [ ]:
train_label

array([4, 4, 4, ..., 1, 2, 2])

### 코드 분석 (3)



```
tfidf = TfidfVectorizer(analyzer='word', sublinear_tf=True, ngram_range=(1, 2), max_features=150000, binary=False)

tfidf.fit(train_text)

train_tf_text = tfidf.transform(train_text).astype('float32')
test_tf_text  = tfidf.transform(test_text).astype('float32')
```


</br>

1. 개념 (참고 : https://mingchin.tistory.com/7)

- **Tf (Term Frequency)** : 특정 단어가 한 문장에 출현하는 횟수  
- **Idf (Inverse of Documnet Frequency)** : df는 특정 단어가 몇 개의 문장에 출현하는지 횟수. 이에 역수를 취한 것이 Idf 
- **Tfidf** :  tf 와 idf를 곱한 것  

**TfidfVectorizer.fit(text)**를 통해 text가 가지고 있는 모든 단어를 BoW *(Bag of Words ; index로는 unique한 단어가 들어가고 value로 그 단어의 빈도가 들어간다)* 로 구성하고, 이 단어들에 대해 Tf-idf 값을 계산한 뒤 각 단어의 인덱스 위치에 Tf-idf 값이 들어간 벡터가 만들어진다.

</br>

2. 파라미터 (참고 : https://chan-lab.tistory.com/27)

- **analyzer** : analyzer = 'word'라고 설정시, 학습의 단위를 단어로 설정
- **sublinear_tf** : sublinear_tf = True 로 설정하면 Tf 값을 'Tf -> 1 + ln(Tf)'로 변경하여 smoothing한다. 지나치게 큰 Tf 값을 갖는 이상치들이 존재할 때 효과적일 수 있다고 한다.
- **ngram_range** : n-gram이라는 것은 단어의 묶음을 말합니다.
이 단어의 묶음을 범위를 설정하는 것이 ngram_range 파라미터. ngram_range = (1, 2)라고 한다면, 단어의 묶음을 1개부터 2개까지 설정하라는 뜻.
- **max_features** : TF-IDF 벡터는 단어사전의 인덱스만큼 feature를 부여받습니다.  tf-idf vector의 최대 feature를 설정.
- **vacabulary** : 어떤 BoW로 벡터화를 진행할 지 직접 지정할 수 있다. (베이스라인 코드엔 없었지만 활용할 수 있을거같음)






In [ ]:
# 코드 참고 : https://mingchin.tistory.com/7

# 각 단어와 맵핑된 인덱스 출력
list(tfidf.vocabulary_.items())[0:10]

[('인천', 61024),
 ('핀란드', 128528),
 ('항공기', 135741),
 ('결항', 2010),
 ('휴가철', 148527),
 ('여행객', 29956),
 ('분통', 16937),
 ('인천 핀란드', 61107),
 ('핀란드 항공기', 128537),
 ('항공기 결항', 135742)]

In [ ]:
# train_text 로부터 각 단어의 빈도수(tfidf)를 기록
# (* RAM 문제로 train 데이터의 10개 문장에 대해서만 시행하였습니다.) 
print(tfidf.fit_transform(train_text[0:10]).toarray())

[[0.        0.        0.        ... 0.2773501 0.2773501 0.       ]
 [0.        0.        0.2773501 ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 ...
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]


In [ ]:
# 문서(문장) 간 유사도도 수치화
# (* RAM 문제로 train 데이터의 10개 문장에 대해서만 시행하였습니다.)
from sklearn.metrics.pairwise import linear_kernel
tfidf_matrix = tfidf.fit_transform(train_text[0:11])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.

8번째 문장과 10번째 문장 간의 유사도 : 0.04127

## 2) LGBM

In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 22.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import re
from konlpy.tag import Okt,Mecab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score,f1_score
from lightgbm import LGBMClassifier

In [4]:
# 형태소 분석기(Okt) 불러오기 
okt=Okt() 

# 조사, 어미, 구두점 제거
def func(text):
    clean = []
    for word in okt.pos(text, stem=True): #어간 추출
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']: #조사, 어미, 구두점 제외 
            clean.append(word[0])
    
    
    return " ".join(clean) 

train['title'] = train['title'].apply(lambda x : func(x))

In [5]:
# tf-idf를 이용한 벡터화
def split(text):
    tokens_ko = text.split()
    return tokens_ko

tfidf_vect = TfidfVectorizer(tokenizer=split)
tfidf_vect.fit(train['title'])
tfidf_matrix_train = tfidf_vect.transform(train['title'])

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
# train/valid 데이터 셋 나누기.
def split_dataset(tfidf,df):
    X_data = tfidf
    y_data = df['topic_idx']

    # stratify=y_data Stratified 기반 분할, train 데이터의 30%를 평가 데이터 셋으로 사용. (70% 데이터 학습에 사용)
    X_train, X_test, y_train, y_test = \
    train_test_split(X_data, y_data, test_size=0.3, random_state=42, stratify=y_data)

    
    return (X_train, X_test, y_train, y_test)

X_train, X_test, y_train, y_test = split_dataset(tfidf_matrix_train,train)